In [1]:
import sys

if ".." not in sys.path:
    sys.path.append("..")

In [2]:
import pickle 

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import cobra

from pprint import pp
from molmass import Formula
from cobra.io import read_sbml_model
from experiments.fast_dFBA import ConstantBounds
from utils.cobra_utils import get_or_create_exchange
from parameters.drawdown import *
from plotting.diagnostic import uptakes_and_secretions, plot_pathway, plot_metabolite_fluxes

In [3]:
MODEL_FILE = "../model/Rpom_05.xml"
BIOMASS_ID = "Rpom_hwa_biomass"
DATA_FILE = "../data/clean/CUE2/dFBA.pkl"


# Load and set up model
model = read_sbml_model(MODEL_FILE)

# Set maintenance to 25 mmol/gDW/h
atpm = model.reactions.get_by_id("ATPM")
atpm.bounds = (25, 25)

ex_glc = model.reactions.get_by_id("EX_glc")
ex_ace = get_or_create_exchange(model, "ACET[e]")

# Load omics data
rna_rel = pd.read_csv("../data/clean/omics/rna-rel.csv")
rna_abs = pd.read_csv("../data/clean/omics/rna-abs.csv")
prot = pd.read_csv("../data/clean/omics/prot.csv")

# Convert reaction_ids column to list
for sheet in [rna_rel, rna_abs, prot]:
    sheet["reaction_ids"] = sheet["reaction_ids"].apply(lambda x: x.split(", ") if isinstance(x, str) else None)

In [4]:
# Get fluxes when running on glucose, acetate
with model:
    ex_glc.bounds = (-10, 0)
    sol_glc = model.optimize()
    mu_glc = sol_glc.objective_value
    model.summary()
    print(f"Glucose growth: {sol_glc.objective_value:.2f}")

with model:
    ex_ace.bounds = (-10, 0)
    sol_ace = model.optimize()
    mu_ace = sol_ace.objective_value
    model.summary()
    print(f"Acetate growth: {sol_ace.objective_value:.2f}")

model_fc = ((sol_ace.fluxes) / mu_ace) / ((sol_glc.fluxes) / mu_glc)

# Get fold-changes, taking the absolute value as we're interested in the magnitude of flux,
# and not necessarily the direction
# (Although maybe direction-switching fluxes are interesting?)
model_abs_fc = (np.abs(sol_ace.fluxes) / mu_ace) / (np.abs(sol_glc.fluxes) / mu_glc)

model_diff = np.abs(sol_ace.fluxes / mu_ace) - np.abs(sol_glc.fluxes / mu_glc)


Glucose growth: 0.76
Acetate growth: 0.20


In [ ]:
# For each reaction in the flux vectors, get the max of the corresponding RNA levels

rna_max = []
rna_max_fc = []  # ace / glc
for rxnid in sol_glc.fluxes.index:
    stem = model.reactions.get_by_id(rxnid).notes["stem"]

    matching = [stem in ids if isinstance(ids, list) else False for ids in rna_abs["reaction_ids"]]
    if sum(matching) == 0:
        rna_max.append(None)
        rna_max_fc.append(None)
    else:
        rna_max.append(rna_abs[matching]["DSS3_glc_mean_abund"].max())
        rna_max_fc.append(rna_abs[matching]["DSS3_ac_mean_abund"].max() / rna_abs[matching]["DSS3_glc_mean_abund"].max())

KeyError: 'stem'

In [ ]:
import plotly.express as px

df = pd.DataFrame(
    [
        {
            "Reaction ID": rxnid,
            "Model flux": abs(flux),
            "RNA": rna
        }
        for (rxnid, flux), rna in zip(sol_glc.fluxes.items(), rna_max)
    ])

fig = px.scatter(
    df, x="Model flux", y="RNA",
    hover_data='Reaction ID',
    # log_x=True,
    # log_y=True
)
fig.show()

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 10, in <module>
NameError: name 'sol_glc' is not defined



In [ ]:
import plotly.express as px

df = pd.DataFrame(
    [
        {
            "Reaction ID": rxnid,
            "Model fold-change (ace/glc)": fc,
            "RNA fold-change (ace/glc)": rna_fc
        }
        for (rxnid, fc), rna_fc in zip(model_abs_fc.items(), rna_max_fc)
        if (not np.isnan(fc)
            and not np.isinf(fc)
            and rna_fc is not None
            and not np.isnan(rna_fc)
            and not np.isinf(rna_fc))
    ])

fig = px.scatter(
    df, x="Model fold-change (ace/glc)", y="RNA fold-change (ace/glc)",
    hover_data='Reaction ID',
    # log_x=True,
    # log_y=True
)
fig.show()

fig.write_html("transcriptome_vs_flux.html")

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 10, in <module>
NameError: name 'model_abs_fc' is not defined



In [ ]:
model.reactions.get_by_id("RXN-8957")

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'model' is not defined



In [ ]:
model.reactions.get_by_id("ACETATE--COA-LIGASE-RXN")

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'model' is not defined



In [ ]:
# acs is always kept online?
# RXN-8957 is ethylmalonyl-CoA pathway, specifically crotonyl-CoA + CO2 + NADPH -> (2S)-ethylmalonyl-CoA + NADP
    # (Major NADP generating step according to FBA)

In [ ]:
# Get all reactions (stems) in the ethylmalonyl-CoA pathway
ethCoA_rxns = set([rxn.notes["stem"] for rxn in model.reactions if "PWY-5741" in rxn.notes.get("pathways", [])])

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2, in <module>
NameError: name 'model' is not defined



In [ ]:
sorted_model_log_fc = df.sort_values("Model fold-change (ace/glc)", ascending=True).dropna()
sorted_data_log_fc = df.sort_values("RNA fold-change (ace/glc)", ascending=True).dropna()

# Drop zeros to avoid bad logs
sorted_model_log_fc = sorted_model_log_fc[sorted_model_log_fc["Model fold-change (ace/glc)"] != 0]
sorted_data_log_fc = sorted_data_log_fc[sorted_data_log_fc["RNA fold-change (ace/glc)"] != 0]

# Take logs
sorted_model_log_fc["Model fold-change (ace/glc)"] = np.log(sorted_model_log_fc["Model fold-change (ace/glc)"])
sorted_data_log_fc["RNA fold-change (ace/glc)"] = np.log(sorted_data_log_fc["RNA fold-change (ace/glc)"])

fig, axs = plt.subplots(2, 1)

N = 20
GAP_SIZE = 10

colors = [str(c) for c in np.linspace(0.7, 1, N)]
for i, rxn in enumerate(sorted_model_log_fc["Reaction ID"].head(N)):
    if rxn in ethCoA_rxns:
        colors[i] = "tab:blue"
axs[0].bar(np.arange(N),
           sorted_model_log_fc["Model fold-change (ace/glc)"].head(N),
           color = colors,
           edgecolor=[str(c) for c in np.linspace(0.5, 0.8, N)])

colors = [str(c) for c in np.linspace(1, 0.7, N)]
for i, rxn in enumerate(sorted_model_log_fc["Reaction ID"].tail(N)):
    if rxn in ethCoA_rxns:
        colors[i] = "tab:blue"
axs[0].bar(np.arange(N + GAP_SIZE, GAP_SIZE + 2 * N),
           sorted_model_log_fc["Model fold-change (ace/glc)"].tail(N),
           color = colors,
           edgecolor=[str(c) for c in np.linspace(0.8, 0.5, N)])

colors = [str(c) for c in np.linspace(0.7, 1, N)]
for i, rxn in enumerate(sorted_data_log_fc["Reaction ID"].head(N)):
    if rxn in ethCoA_rxns:
        colors[i] = "tab:blue"
axs[1].bar(np.arange(N),
           sorted_data_log_fc["RNA fold-change (ace/glc)"].head(N),
           color=[
               "tab:blue" if rxn in ethCoA_rxns else str(np.linspace(0.7, 1, N)[i])
               for i, rxn in enumerate(sorted_data_log_fc["Reaction ID"].head(N))
           ],
           edgecolor=[str(c) for c in np.linspace(0.5, 0.8, N)])

colors = [str(c) for c in np.linspace(1, 0.7, N)]
for i, rxn in enumerate(sorted_data_log_fc["Reaction ID"].tail(N)):
    if rxn in ethCoA_rxns:
        colors[i] = "tab:blue"
axs[1].bar(np.arange(N + GAP_SIZE, GAP_SIZE + 2 * N),
           sorted_data_log_fc["RNA fold-change (ace/glc)"].tail(N),
           color=[
               "tab:blue" if rxn in ethCoA_rxns else str(np.linspace(1, 0.7, N)[i])
               for i, rxn in enumerate(sorted_data_log_fc["Reaction ID"].tail(N))
           ],
           edgecolor=[str(c) for c in np.linspace(0.8, 0.5, N)])

# Baselines
axs[0].hlines(0, -1, 2*N + GAP_SIZE, color="0.2", linewidth=1)
axs[1].hlines(0, -1, 2*N + GAP_SIZE, color="0.2", linewidth=1)

# Hide spines, x ticks
for ax in axs:
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.spines.bottom.set_visible(False)
    ax.set_xticks([])

# ax.set_xlim(0, 3 * N + 1)
# ax.set_ylim(sorted_model_log_fc["Model fold-change (ace/glc)"].min(),
            # sorted_model_log_fc["Model fold-change (ace/glc)"].max())

axs[0].text(N, 0, "Model", ha="center", va="bottom", color="0.2")
axs[1].text(N + 5, 0, "Data", ha="center", va="bottom", color="0.2")

axs[0].set_ylabel("Log FC (ace/glc)")
axs[1].set_ylabel("Log FC (ace/glc)")

fig.subplots_adjust(hspace=0.1)
fig.tight_layout()
fig.set_size_inches(8, 4)

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
sorted_model_log_fc.head(20)

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'sorted_model_log_fc' is not defined



In [ ]:
sorted_data_log_fc.head(20)

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'sorted_data_log_fc' is not defined



In [ ]:
sorted_model_log_fc["Model fold-change (ace/glc)"].head(N)

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'sorted_model_log_fc' is not defined



In [ ]:
df.sort_values("Model fold-change (ace/glc)", ascending=True).dropna()

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
rna_abs.iloc[:, 7:]

Traceback (most recent call last):
  File "/home/mica/.vscode-server/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'rna_abs' is not defined

